In [1]:
!pip install groq

In [2]:
import requests
import random
from groq import Groq

In [3]:
def get_imdb_fan_favorites():
    url = "https://imdb188.p.rapidapi.com/api/v1/getFanFavorites"
    querystring = {"country": "US"}
    headers = {
        "x-rapidapi-key": "b433dcc36fmsh6c6388a93d045c7p1f429cjsn4cb93cd655c4",
        "x-rapidapi-host": "imdb188.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)
    if response.status_code == 200:
        return response.json().get('data', {}).get('list', [])
    else:
        print("Error fetching IMDB data:", response.status_code)
        return []

In [4]:
def get_weather_forecast(location_id="53286"):
    url = f"http://dataservice.accuweather.com/forecasts/v1/daily/1day/{location_id}?apikey=aOSV4TZGheO7mWpDWnYy8xlfjhVhnHbf"
    response = requests.get(url)
    if response.status_code == 200:
        forecast = response.json()
        weather_text = forecast['Headline']['Text']
        temperature = forecast['DailyForecasts'][0]['Temperature']
        day_forecast = forecast['DailyForecasts'][0]['Day']
        return {
            "text": weather_text,
            "temperature": temperature,
            "day_forecast": day_forecast
        }
    else:
        print("Error fetching weather data:", response.status_code)
        return {}

In [5]:
def generate_movie_recommendation(weather_info, movies_list):
    client = Groq(api_key="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")
    weather_text = weather_info.get("text", "")
    temperature_min = weather_info["temperature"]["Minimum"]["Value"]
    temperature_max = weather_info["temperature"]["Maximum"]["Value"]

    prompt = f"The weather forecast says: '{weather_text}'. The temperature will be between {temperature_min}F and {temperature_max}F. "
    prompt += "Based on this weather, suggest a few relaxing movies from the list below:\n\n"
    for movie in movies_list:
        title = movie['originalTitleText']['text']
        rating = movie['ratingsSummary']['aggregateRating']
        prompt += f"- {title} (Rating: {rating})\n"

    try:
        completion = client.chat.completions.create(
            model="llama-3.1-70b-versatile",
            messages=[
                {"role": "system", "content": prompt}
            ],
            temperature=1,
            max_tokens=1024,
            top_p=1,
            stop=None
        )

        # Extract and print the recommendation text
        recommendation_text = completion.choices[0].message.content
        print(recommendation_text)

    except Exception as e:
        print("Error during completion request:", str(e))

In [8]:
def main():
    # Step 1: Fetch IMDB fan favorite movies
    movies_list = get_imdb_fan_favorites()
    if not movies_list:
        print("No movies found.")
        return

    # Step 2: Fetch AccuWeather forecast for Vancouver
    weather_info = get_weather_forecast()
    if not weather_info:
        print("No weather data found.")
        return

    # Step 3: Generate movie recommendations based on weather
    print("Generating movie recommendations based on the weather...\n")
    generate_movie_recommendation(weather_info, movies_list)

# Run the application
main()

Generating movie recommendations based on the weather...

Based on the rainy weather forecast, I'd recommend a few relaxing movies from your list. Here are some suggestions:

1. **Shawshank Redemption** (Rating: 9.3) - A highly-rated film about hope, redemption, and friendship, perfect for a cozy evening on a rainy day.
2. **The Lord of the Rings: The Fellowship of the Ring** (Rating: 8.9) - A classic fantasy film that will transport you to a different world, providing an escape from the gloomy weather outside.
3. **Forrest Gump** (Rating: 8.8) - A heartwarming, feel-good movie with a simple yet inspiring story that's easy to get lost in on a rainy day.
4. **Inception** (Rating: 8.8) - A thought-provoking sci-fi film that will keep your mind engaged while you relax and unwind.
5. **The Matrix** (Rating: 8.7) - A classic, intelligent sci-fi film that explores the idea of a simulated reality, perfect for a rainy evening.

These movies offer a mix of engaging stories, memorable characters